In [ ]:
# from sklearn.metrics import log_loss
lgb_predict_a = lgb_clf.predict_proba(test_data[features])[:, 1]
# lgb_predict_b = lgb_clf_b.predict_proba(test_data[features])[:, 1]
xgb_predict = xgb_a.predict(dtest)

# result = lgb_predict * 0.5 + xgb_predict * 0.5

avg_predict = lgb_predict_a * 0.5 + xgb_predict * 0.5

loss_test = log_loss(test_data[target], avg_predict)
loss_test


In [ ]:
train_data = all_data[(all_data.day >= 18) & (all_data.day <= 24)]
test_data = all_data[all_data.day == 25]

dtrain = xgb.DMatrix(train_data[features], train_data[target])
dtest = xgb.DMatrix(test_data[features], test_data[target])

watchlist = [(dtrain, 'train')]

params = {
    'n_estimators': 100,
    'max_depth': 3,
    'eta': 0.1,
    'eval_metric': 'logloss',
    'objective': 'binary:logistic',
    'subsample': 1.0,
    'colsample_bytree': 0.7,
#     'random_state': 1123,
#     'min_child_weight': 10
    #'scale_pos_weight':0.5
}

xgb_a = xgb.train(params, dtrain,
                  num_boost_round=250,
                  early_stopping_rounds=20,
                  evals=watchlist,
                  verbose_eval=10)


lgb_a = LGBMClassifier(n_estimators=200, max_depth=3, 
#                         num_leaves=50, 
#                        learning_rate=0.1, 
#                        seed=0, nthread=24, subsample=0.8, colsample_bytree=0.9, reg_lambda=0.005, 
                      )

cate_features = ['user_gender_id', 'user_occupation_id']

lgb_a.fit(train_data[features], train_data['is_trade'],
          #         eval_set=[(train_x.loc[train_x.clickDay>25,feature_group_A],train_x.loc[train_x.clickDay>25,['label']])],
          feature_name=features,
          categorical_feature=cate_features,
          verbose=50,
          )

lgb_predict_a = lgb_a.predict_proba(test_data[features])[:, 1]
xgb_predict_a = xgb_a.predict(dtest)

avg_predict = lgb_predict_a * 0.5 + xgb_predict_a * 0.5

test_data['predicted_score'] = avg_predict

test_data[['instance_id', 'predicted_score']].to_csv(
    '../submission/20180403.txt', index=False, sep=' ')


lgb_predict_a = lgb_a.predict_proba(train_data[features])[:, 1]
xgb_predict_a = xgb_a.predict(dtrain)

avg_predict = lgb_predict_a * 0.5 + xgb_predict_a * 0.5

loss_train = log_loss(train_data[target], avg_predict)

loss_train


In [ ]:
lgb_predict_a = lgb_a.predict_proba(train_data[features])[:, 1]
xgb_predict_a = xgb_a.predict(dtrain)

avg_predict = lgb_predict_a * 0.5 + xgb_predict_a * 0.5

loss_train = log_loss(train_data[target], lgb_predict_a)

loss_train